<a href="https://colab.research.google.com/github/jtao/VIST271/blob/main/Image_Composition_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Composition Examples

Jian Tao, Texas A&M University

Feb 24, 2022

The later part of this Jupyter notebook on COCO Data Set is based on the example at [virafpatrawala/Synthetic-Image-Datasets](https://github.com/virafpatrawala/Synthetic-Image-Datasets/blob/master/SyntheticImageData_ImageComposition_Demo.ipynb).


In [ ]:
cat_path = "https://github.com/jtao/VIST271/raw/main/images/cat.png"
cat_file = "cat.png"

In [ ]:
oak_path = "https://github.com/jtao/VIST271/raw/main/images/post_oak.jpg"
oak_file = "post_oak.jpg"

# Read Image

We first download an image from a URL and then open it with the Image.open function in PIL.

In [ ]:
from PIL import Image, ImageFilter, ImageDraw, ImageFont, ImageOps
import numpy as np
import skimage.io as io
from matplotlib import pyplot as plt

In [ ]:
import urllib.request
from PIL import Image
urllib.request.urlretrieve(cat_path, cat_file)
cat = Image.open(cat_file)
cat

In [ ]:
cat.size

PNG images usually have four channels. Three color channels for red, green and blue, and the fourth channel is for transparency.

In [ ]:
cat.mode

In [ ]:
cat.info

In [ ]:
from PIL import ImageEnhance
def change_opacity(image, opacity):
  if opacity < 0 and opacity > 1:
    print ("opacity value must be between 0 and 1!")
    return
  if image.mode != 'RGBA':
      image = image.convert('RGBA')
  else:
      image = image.copy()
  alpha = image.split()[3]
  alpha = ImageEnhance.Brightness(alpha).enhance(opacity)
  image.putalpha(alpha)
  return image

In [ ]:
def image_grid(imgs, rows, cols):
  if len(imgs) != rows*cols:
    print ("number of images must be equal to rows * cols!")
    return
  w, h = imgs[0].size
  grid = Image.new('RGBA', size=(cols*w, rows*h))
  grid_w, grid_h = grid.size
    
  for i, img in enumerate(imgs):
    grid.paste(img, box=(i%cols*w, i//cols*h))
  return grid

In [ ]:
myimages = []
for i in range(9):
  opacity = i/8
  myimages.append(change_opacity (cat, opacity))
image_grid(myimages, 3, 3)

In [ ]:
urllib.request.urlretrieve(oak_path, oak_file)
oak = Image.open(oak_file)
oak

In [ ]:
oak.mode

In [ ]:
oak = oak.convert("RGBA")

In [ ]:
oak.mode

In [ ]:
oak.size

In [ ]:
cat.size

In [ ]:
cat = cat.resize((800,600))

# Alpha Composition

In [ ]:
Image.alpha_composite(oak, cat)

# Blend with a constant alpha $$ z = \alpha x + (1-\alpha) y$$

In [ ]:
Image.blend(cat, oak, 0.7)

# Composition with Different Masks

In [ ]:
mask = Image.new("L", oak.size, 0)
draw = ImageDraw.Draw(mask)
draw.ellipse((200, 200, 400, 400), fill=255)

In [ ]:
mask

In [ ]:
Image.composite(cat, oak, mask)

In [ ]:
mask_blur = mask.filter(ImageFilter.GaussianBlur(20))
mask_blur

In [ ]:
Image.composite(cat, oak, mask_blur)

In [ ]:
vizlogo_path = "https://github.com/jtao/VIST271/raw/main/images/viz_logo.png"
vizlogo = "viz_logo.png"
urllib.request.urlretrieve(vizlogo_path, vizlogo)
mask = Image.open(vizlogo).convert('L').resize(oak.size)
Image.composite(oak, cat, mask)

# Animation GIFs

In [ ]:
from PIL import Image, ImageDraw

images = []

width = cat.width
center = width // 2
max_radius = int(center * 1.5)
step = 8

for i in range(0, max_radius, step):
    im = Image.new('L', cat.size, 0)
    draw = ImageDraw.Draw(im)
    draw.ellipse((center - i, center - i, center + i, center + i), fill=255)


    im = Image.composite(oak, cat, im)
    images.append(im)


images[0].save('cat_oak.gif', save_all=True, append_images=images[1:], 
               optimize=True, duration=20, loop=0)

# Common Objects in Context (COCO) Dataset

[COCO](https://cocodataset.org) is a large-scale object detection, segmentation, and captioning dataset. COCO has several features:

* Object segmentation
* Recognition in context
* Superpixel stuff segmentation
* 330K images (>200K labeled)
* 1.5 million object instances
* 80 object categories
* 91 stuff categories
* 5 captions per image
* 250,000 people with keypoints



In [ ]:
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -o annotations_trainval2017.zip

In [ ]:
from pycocotools.coco import COCO
dataDir='.'
dataType='val'
annFile='{}/annotations/instances_{}2017.json'.format(dataDir,dataType)
coco=COCO(annFile)
catIDs = coco.getCatIds(catNms=['cat'])
imgIds = coco.getImgIds(catIds=catIDs)
images = coco.loadImgs(imgIds)

In [ ]:
len(images)

In [ ]:
# downlaod all the files (it will take a long while for training images, validation images are ok)
for im in images:
  urllib.request.urlretrieve(im['coco_url'], im['file_name'])

In [ ]:
nrows=5
ncols=10
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
for i in range(nrows):
  for j in range(ncols):
    I=io.imread(images[i*ncols+j]['file_name'])/255.0
    axes[i,j].imshow(I)
    axes[i,j].set_axis_off()    
plt.show()

In [ ]:
# load and display instance annotations
def image_ann(im, ann=True):
  I=io.imread(im['file_name'])
  plt.imshow(I)
  plt.axis('off')
  if ann:
    annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIDs, iscrowd=None)
    anns = coco.loadAnns(annIds)
    coco.showAnns(anns)

In [ ]:
## Get the Binary Mask
def binary_mask(im):
  mask = np.zeros((im['height'],im['width']))
  annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIDs, iscrowd=None)
  anns = coco.loadAnns(annIds)
  for i in range(len(anns)):
      mask = np.maximum(coco.annToMask(anns[i]), mask)
  return mask
  # plt.imshow(mask)
  # plt.axis('off')
  # plt.show()

In [ ]:
image_ann(images[5], ann=True)

In [ ]:
binary_mask(images[5])

In [ ]:
# load and display instance annotations
def extract_cat(im):
  I=io.imread(im['file_name'])/255.0
  mask = np.zeros((im['height'],im['width']))
  annIds = coco.getAnnIds(imgIds=im['id'], catIds=catIDs, iscrowd=None)
  anns = coco.loadAnns(annIds)
  for i in range(len(anns)):
      mask = np.maximum(coco.annToMask(anns[i]), mask)

  mask = mask.reshape(I.shape[0], I.shape[1], 1)

  # Cut out the object from its original image
  foreground = mask*I
  return foreground
  # plt.imshow(foreground)
  # plt.axis('off')
  # plt.show()

In [ ]:
images[4]

In [ ]:
image_ann(images[0], ann=False)

In [ ]:
extract_cat(images[0])

In [ ]:
# Visualize the background
plt.imshow(oak)
plt.axis('off')
plt.show()

In [ ]:
import skimage.transform as transform
import numpy as np
def compose(foreground, mask, background):
    # resize background
    background = background.convert("RGB")
    background = np.array(background)
    background = transform.resize(background, foreground.shape[:2])

    # Subtract the foreground area from the background
    background = background*(1 - mask.reshape(foreground.shape[0], foreground.shape[1], 1))

    # Finally, add the foreground
    composed_image = background + foreground
    
    return composed_image

In [ ]:
composed_image = compose(extract_cat(images[0]), binary_mask(images[0]), oak)
plt.imshow(composed_image)
plt.axis('off')
plt.show()    

In [ ]:
nrows=5
ncols=10
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*5))
for i in range(nrows):
  for j in range(ncols):
    composed_image = compose(extract_cat(images[i*ncols + j]), binary_mask(images[i*ncols +j]), oak)
    axes[i,j].imshow(composed_image)
    axes[i,j].set_axis_off()    
plt.show()